In [1]:
import numpy as np

In [2]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import os
import subprocess
import requests
import boto3
import s3fs
import pandas as pd
import numpy as np
import xarray as xr
import pyproj
from pyproj import Proj
from shapely.ops import transform
from cartopy import crs
import time
import dask
from pathlib import Path


In [5]:
from dask.distributed import Client
try:
    client.close()
except Exception as e:
    print(e)
    
client = Client("tcp://127.0.0.1:33323")
client

name 'client' is not defined


<Client: 'tcp://127.0.0.1:33323' processes=8 threads=32, memory=123.94 GiB>

In [6]:
def gen_json(args):
    u, var_name = args
    print(u, var_name)
    
    so = dict(
        mode="rb", anon=False, default_fill_cache=False,
        default_cache_type="none"
    )
    try:
        P = Path(f'/home/jpluser/jsons/{var_name}');
        print(f'P: {str(P)}')
        print(P.mkdir(exist_ok=True))
        
        with fsspec.open(u, **so) as inf:
            h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
            with open(f"/home/jpluser/jsons/{var_name}/{u.split('/')[-1]}.json", 'wb') as outf:
               outf.write(ujson.dumps(h5chunks.translate()).encode())
    except Exception as e:
        print(e)

In [7]:
fs = fsspec.filesystem('s3', anon=False)

In [9]:
sassie_dataset_paths = fs.glob("s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/*AVG*")
from pprint import pprint
pprint(sassie_dataset_paths)

sassie_dataset_names = [p.split('/')[-1] for p in sassie_dataset_paths]
pprint(sassie_dataset_names)

['ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_SLT_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_TH_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVxHEFF_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVxSNOW_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVx_SLT_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVx_TH_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVyHEFF_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVySNOW_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVy_SLT_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVy_TH_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/DFrE_SLT_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/DFrE_TH_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/DFrI_SLT_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/DFrI_TH_AVG_DAILY',
 'ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ETAN_AVG_DAILY',
 'ecco-processed-da

In [10]:
def make_combined_json(sassie_dataset_path):
    
    var_name = sassie_dataset_path.split('/')[-1]
    print(var_name)
    print(sassie_dataset_path)
    data_urls = ['s3://' + f for f in fs.glob(f"s3://{sassie_dataset_path}/*.nc")]#SIheff_AVG_DAILY/*nc")]
    print(data_urls[0])
    print(data_urls[-1])
    
    gen_json([data_urls[0], var_name])
    
    st = time.time()
    dask.compute(*[dask.delayed(gen_json)([u, var_name]) for u in data_urls]);
    print('gen_json ', time.time()-st)
    
    
    json_list = sorted(glob(f"/home/jpluser/jsons/{var_name}/*.json"))
    m_list = []
    
    st = time.time()
    for js in tqdm(json_list):
        with open(js) as f:
            m_list.append(fsspec.get_mapper("reference://", 
                fo=ujson.load(f), remote_protocol="s3",
                remote_options={"anon": False}))
    
    json_list = sorted(glob(f'/home/jpluser/jsons/{var_name}/*.json'))
    print('m_list make ', time.time()-st)
    
    st = time.time()
    mzz = MultiZarrToZarr(
        json_list,
        remote_protocol="s3",
        remote_options={'anon':False},
        concat_dims='time',
        inline_threshold=0
    )
    print('multi zarr to zarr ', time.time()-st)
    
    st = time.time()
    mzz.translate(f'/home/jpluser/{var_name}_combined.json')
    print('mzz translate ', time.time()-st)

In [11]:
sassie_dataset_path = sassie_dataset_paths[42]
for s in sassie_dataset_paths:
    make_combined_json(s)

ADVr_SLT_AVG_DAILY
ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_SLT_AVG_DAILY
s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_SLT_AVG_DAILY/ADVr_SLT_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc
s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_SLT_AVG_DAILY/ADVr_SLT_day_mean_2021-02-07_ECCO_SASSIE_V1_HH_llc1080.nc
s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_SLT_AVG_DAILY/ADVr_SLT_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc ADVr_SLT_AVG_DAILY
P: /home/jpluser/jsons/ADVr_SLT_AVG_DAILY
None
gen_json  1139.5016236305237


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2548/2548 [00:04<00:00, 607.25it/s]


m_list make  4.206785440444946
multi zarr to zarr  3.647804260253906e-05
mzz translate  741.022935628891
ADVr_TH_AVG_DAILY
ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_TH_AVG_DAILY
s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_TH_AVG_DAILY/ADVr_TH_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc
s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_TH_AVG_DAILY/ADVr_TH_day_mean_2021-02-07_ECCO_SASSIE_V1_HH_llc1080.nc
s3://ecco-processed-data/SASSIE/N1/V1/HH/NETCDF/ADVr_TH_AVG_DAILY/ADVr_TH_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc ADVr_TH_AVG_DAILY
P: /home/jpluser/jsons/ADVr_TH_AVG_DAILY
None
gen_json  1156.7277846336365


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2548/2548 [00:04<00:00, 607.94it/s]


m_list make  4.2032599449157715
multi zarr to zarr  2.5033950805664062e-05


KeyboardInterrupt: 